In [3]:
from operator import rshift
import cv2 as cv
import numpy as np
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
# right eyes indices
RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

LIPS=[ 61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78 ]
LOWER_LIPS =[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS=[ 185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 

cap = cv.VideoCapture(0)
with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
) as face_mesh:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv.flip(frame, 1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            # print(results.multi_face_landmarks[0].landmark)
            mesh_points = np.array(
                [np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            # print(mesh_points.shape)
            # cv.polylines(frame, [mesh_points[LEFT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)
            # cv.polylines(frame, [mesh_points[RIGHT_IRIS]], True, (0,255,0), 1, cv.LINE_AA)
            
            # 입 윤곽 그리기
            cv.polylines(frame, [mesh_points[LIPS]], isClosed=True, color=(0, 255, 0), thickness=1, lineType=cv.LINE_AA)
            cv.polylines(frame, [mesh_points[LOWER_LIPS]], isClosed=True, color=(0, 0, 255), thickness=1, lineType=cv.LINE_AA)
            cv.polylines(frame, [mesh_points[UPPER_LIPS]], isClosed=True, color=(255, 0, 0), thickness=1, lineType=cv.LINE_AA)

            (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
            center_left = np.array([l_cx, l_cy], dtype=np.int32)
            center_right = np.array([r_cx, r_cy], dtype=np.int32)
            cv.circle(frame, center_left, int(l_radius), (255, 0, 255), 1, cv.LINE_AA)
            cv.circle(frame, center_right, int(r_radius), (255, 0, 255), 1, cv.LINE_AA)

        cv.imshow('img', frame)
        key = cv.waitKey(1)
        if key == ord('q'):
            break
cap.release()
cv.destroyAllWindows()

In [6]:

import cv2 as cv 
import numpy as np
import mediapipe as mp 
mp_face_mesh = mp.solutions.face_mesh

# left eyes indices
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ] 

# irises Indices list
LEFT_IRIS = [474,475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

LIPS=[ 61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78 ]
LOWER_LIPS =[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS=[ 185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 

cap = cv.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv.flip(frame, 1)

        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        mask = np.zeros((img_h, img_w), dtype=np.uint8)

        if results.multi_face_landmarks:
            # print((results.multi_face_landmarks[0]))

            # [print(p.x, p.y, p.z ) for p in results.multi_face_landmarks[0].landmark]
            
            mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) 
            for p in results.multi_face_landmarks[0].landmark])
            
            # 입술을 마스크에 그리기
            cv.polylines(mask, [mesh_points[LIPS]], isClosed=True, color=(255, 255, 255), thickness=1, lineType=cv.LINE_AA)
            cv.fillPoly(mask, [mesh_points[LIPS]], color=(255, 255, 255)) # 내부를 채우기
            
            (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
            center_left = np.array([l_cx, l_cy], dtype=np.int32)
            center_right = np.array([r_cx, r_cy], dtype=np.int32)
            cv.circle(frame, center_left, int(l_radius), (0,255,0), 2, cv.LINE_AA)
            cv.circle(frame, center_right, int(r_radius), (0,255,0), 2, cv.LINE_AA)
            
            # drawing on the mask 
            cv.circle(mask, center_left, int(l_radius), (255,255,255), -1, cv.LINE_AA)
            cv.circle(mask, center_right, int(r_radius), (255,255,255), -1, cv.LINE_AA)
            
        cv.imshow('Mask', mask)     
        cv.imshow('img', frame)
        
        key = cv.waitKey(1)
        if key ==ord('q'):
            break
cap.release()
cv.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
drawing_spec = mp_drawing.DrawingSpec(thickness=0.5, circle_radius=1)


def getLandmarks(image):
    face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = face_mesh.process(image)
    landmarks = []
    if results != None:
        landmarks = results.multi_face_landmarks[0].landmark

    return landmarks, results


# Draw the face mesh annotations on the image.
def drawFaceMesh(image, results):
    image.flags.writeable = True
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            #         print('face landmarks', face_landmarks)
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACE_CONNECTIONS,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)

    cv2.imshow('MediaPipe FaceMesh', image)


# Crop the right eye region
def getRightEye(image, landmarks):
    eye_top = int(landmarks[263].y * image.shape[0])
    eye_left = int(landmarks[362].x * image.shape[1])
    eye_bottom = int(landmarks[374].y * image.shape[0])
    eye_right = int(landmarks[263].x * image.shape[1])
    right_eye = image[eye_top:eye_bottom, eye_left:eye_right]
    return right_eye


# Get the right eye coordinates on the actual -> to visualize the bbox
def getRightEyeRect(image, landmarks):
    eye_top = int(landmarks[385].y * image.shape[0])  # 257
    eye_left = int(landmarks[362].x * image.shape[1])
    eye_bottom = int(landmarks[380].y * image.shape[0])  # 374
    eye_right = int(landmarks[263].x * image.shape[1])

    cloned_image = image.copy()
    cropped_right_eye = cloned_image[eye_top:eye_bottom, eye_left:eye_right]
    h, w, _ = cropped_right_eye.shape
    x = eye_left
    y = eye_top
    return x, y, w, h


def getLeftEye(image, landmarks):
    eye_top = int(landmarks[159].y * image.shape[0])
    eye_left = int(landmarks[33].x * image.shape[1])
    eye_bottom = int(landmarks[145].y * image.shape[0])
    eye_right = int(landmarks[133].x * image.shape[1])
    left_eye = image[eye_top:eye_bottom, eye_left:eye_right]
    return left_eye


def getLeftEyeRect(image, landmarks):
    # eye_left landmarks (27, 23, 130, 133) ->? how to utilize z info
    eye_top = int(landmarks[159].y * image.shape[0])
    eye_left = int(landmarks[33].x * image.shape[1])
    eye_bottom = int(landmarks[145].y * image.shape[0])
    eye_right = int(landmarks[133].x * image.shape[1])

    cloned_image = image.copy()
    cropped_left_eye = cloned_image[eye_top:eye_bottom, eye_left:eye_right]
    h, w, _ = cropped_left_eye.shape

    x = eye_left
    y = eye_top
    return x, y, w, h


# For webcam input:
#drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
success, image = cap.read()
imgb = np.zeros_like(image, dtype="uint8")

# status marking for current state
sleep = 0
drowsy = 0
active = 0
status = ""
color = (0, 0, 0)
rightEyeWidth = 0
rightEyeHeight = 0
leftEyeWidth, leftEyeHeight = 0, 0

with mp_face_mesh.FaceMesh(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        imgb = np.zeros_like(image, dtype="uint8")
        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        # image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        # image.flags.writeable = False
        # results = face_mesh.process(image)

        # Convert the BGR image to RGB before processing.
        landmarks, results = getLandmarks(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        image.flags.writeable = True
        rightEyeImg = getRightEye(image, landmarks)
        rightEyeHeight, rightEyeWidth, _ = rightEyeImg.shape

        xRightEye, yRightEye, rightEyeWidth, rightEyeHeight = getRightEyeRect(image, landmarks)
        cv2.rectangle(image, (xRightEye, yRightEye), (xRightEye + rightEyeWidth, yRightEye + rightEyeHeight),
                      (200, 21, 36), 2)

        # LEFT EYE
        leftEyeImg = getLeftEye(image, landmarks)
        leftEyeHeight, leftEyeWidth, _ = leftEyeImg.shape

        xLeftEye, yLeftEye, leftEyeWidth, leftEyeHeight = getLeftEyeRect(image, landmarks)
        cv2.rectangle(image, (xLeftEye, yLeftEye), (xLeftEye + leftEyeWidth, yLeftEye + leftEyeHeight), (200, 21, 36),
                      2)

        rightEyeAspectRatio = (rightEyeHeight) / (rightEyeWidth)
        leftEyeAspectRatio = (leftEyeHeight) / (leftEyeWidth)

        eyeAspectRatio = (leftEyeAspectRatio + rightEyeAspectRatio) / 2
        print(eyeAspectRatio)
        # Now judge what to do for the eye blinks
        if (eyeAspectRatio < 0.2):
            sleep += 1
            drowsy = 0
            active = 0
            if (sleep > 6):
                status = "SLEEPING !!!"
                color = (255, 0, 0)

        elif (eyeAspectRatio >= 0.2 and eyeAspectRatio < 0.3):
            sleep = 0
            active = 0
            drowsy += 1
            if (drowsy > 6):
                status = "Drowsy !"
                color = (0, 0, 255)

        else:
            drowsy = 0
            sleep = 0
            active += 1
            if (active > 6):
                status = "Active :)"
                color = (0, 255, 0)

        cv2.putText(image, status, (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)
        cv2.imshow('MediaPipe FaceMesh', image)

        if cv2.waitKey(5):
            break
        if key ==ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

0.3118279569892473
0.3064516129032258
0.3064516129032258
0.3064516129032258
0.27419354838709675
0.27419354838709675
0.2903225806451613
0.29569892473118276
0.26236559139784943
0.26236559139784943
0.26236559139784943
0.26236559139784943
0.26236559139784943
0.27903225806451615
0.2629032258064516
0.2701612903225806
0.2701612903225806
0.2545362903225806
0.2540322580645161
0.2540322580645161
0.2540322580645161
0.2540322580645161
0.2206896551724138
0.2206896551724138
0.23333333333333334
0.23333333333333334
0.23010752688172043
0.259375
0.28420467185761955
0.27903225806451615
0.29516129032258065
0.27903225806451615
0.27903225806451615
0.27903225806451615
0.27903225806451615
0.259375
0.25083426028921024
0.27903225806451615
0.26236559139784943
0.26236559139784943
0.25083426028921024
0.25083426028921024
0.25083426028921024
0.25083426028921024
0.26696329254727474
0.26696329254727474
0.27903225806451615
0.26807563959955505
0.26807563959955505
0.26807563959955505
0.26807563959955505
0.268075639599555

0.2940354147250699
0.2987179487179487
0.3017543859649123
0.31846846846846844
0.31491228070175437
0.31491228070175437
0.31491228070175437
0.2882882882882883
0.2882882882882883
0.33513513513513515
0.33513513513513515
0.31491228070175437
0.32123411978221417
0.31805019305019305
0.31375661375661373
0.28736263736263734
0.30164835164835163
0.3351648351648352
0.30164835164835163
0.30164835164835163
0.33235294117647063
0.328125
0.34081346423562414
0.34081346423562414
0.3456521739130435
0.353494623655914
0.3208333333333333
0.3375
0.2797805642633229
0.2728635682158921
0.2728635682158921
0.2797805642633229
0.2797805642633229
0.3045977011494253
0.35082458770614694
0.34247648902821315
0.3284072249589491
0.325
0.325
0.3111658456486043
0.3111658456486043
0.34247648902821315
0.34247648902821315
0.3167701863354037
0.33358320839580213
0.3045977011494253
0.3254310344827586
0.33358320839580213
0.3254310344827586
0.32999999999999996
0.30641025641025643
0.30641025641025643
0.33518518518518514
0.3518518518518

0.33034647550776586
0.3526785714285714
0.3414351851851852
0.34647550776583036
0.33034647550776586
0.337468982630273
0.3118279569892473
0.34647550776583036
0.3414351851851852
0.33034647550776586
0.33034647550776586
0.33034647550776586
0.33034647550776586
0.33034647550776586
0.34647550776583036
0.29569892473118276
0.2980769230769231
0.2980769230769231
0.25752314814814814
0.2475186104218362
0.1935096153846154
0.178125
0.178125
0.18934081346423562
0.1781524926686217
0.1715542521994135
0.1781524926686217
0.1781524926686217
0.18682065217391303
0.18682065217391303
0.1514726507713885
0.1514726507713885
0.21375
0.19175627240143367
0.18526785714285715
0.1867816091954023
0.18778801843317972
0.20402298850574713
0.20402298850574713
0.18409343715239154
0.18409343715239154
0.18333333333333335
0.18333333333333335
0.18333333333333335
0.18620689655172415
0.2
0.1806451612903226
0.1806451612903226
0.1867816091954023
0.1867816091954023
0.18333333333333335
0.18333333333333335
0.2206896551724138
0.2013348164

0.2367741935483871
0.2367741935483871
0.2367741935483871
0.2367741935483871
0.225
0.225
0.24260752688172044
0.2367741935483871
0.22064516129032258
0.2167741935483871
0.24260752688172044
0.24000000000000002
0.24260752688172044
0.24583333333333335
0.24583333333333335
0.2367741935483871
0.2367741935483871
0.24000000000000002
0.24000000000000002
0.24000000000000002
0.26
0.26
0.26
0.2367741935483871
0.22064516129032258
0.2367741935483871
0.2367741935483871
0.24260752688172044
0.22647849462365593
0.22647849462365593
0.24260752688172044
0.24583333333333335
0.24583333333333335
0.24583333333333335
0.24583333333333335
0.24583333333333335
0.24583333333333335
0.24583333333333335
0.24583333333333335
0.20064516129032256
0.20064516129032256
0.20564516129032256
0.20564516129032256
0.22333333333333333
0.225
0.2367741935483871
0.22
0.22
0.24583333333333335
0.24583333333333335
0.2367741935483871
0.2367741935483871
0.20564516129032256
0.20564516129032256
0.2033333333333333
0.22333333333333333
0.2033333333

0.349833147942158
0.37083333333333335
0.3602150537634409
0.3602150537634409
0.3602150537634409
0.3832035595105673
0.33813364055299544
0.33813364055299544
0.3052083333333333
0.3052083333333333
0.33259176863181317
0.34879032258064513
0.3649193548387097
0.3602150537634409
0.3548387096774194
0.3548387096774194
0.3548387096774194
0.3548387096774194
0.3602150537634409
0.3333333333333333
0.31129032258064515
0.33870967741935487
0.3333333333333333
0.328494623655914
0.3333333333333333
0.3
0.3
0.3
0.3123655913978495
0.3118279569892473
0.31666666666666665
0.29569892473118276
0.3
0.3333333333333333
0.3
0.32258064516129037
0.32258064516129037
0.2946236559139785
0.32258064516129037
0.32258064516129037
0.32258064516129037
0.3170362903225806
0.32258064516129037
0.32258064516129037
0.32258064516129037
0.32258064516129037
0.3336693548387097
0.3336693548387097
0.32258064516129037
0.33316532258064513
0.32258064516129037
0.32258064516129037
0.32258064516129037
0.2618279569892473
0.24516129032258066
0.258064

In [4]:
pip install pyautogui

     ---------------------------------------- 61.2/61.2 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 168.2/168.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): fini

AttributeError: module 'mediapipe.python.solutions' has no attribute 'iris'